In [ ]:
import pandas as pd
import io
import requests
import datetime
import plotly.express as px

## Get the Timeseries Data

Add documentation about the time series here.

In [ ]:
from functions import get_time_series

confirmed, deaths, recovered, time_series_dates = get_time_series(local=True)

## Get all the Daily Reports 

Add documentation about these reports here.

In [ ]:
from functions import get_daily_reports

daily_report_data, daily_report_dates = get_daily_reports(local=True)

## All Done Gathering Data.

Let's merge the time_series_data into one dataframe

In [ ]:
from model import data_by_area
from data import us_confirmed 

data = data_by_area(area='Nebraska', col='State', df=us_confirmed)
data.plot()

In [ ]:

def data_by_area(area='US', col='Country/Region', df=None):
	from data import time_series_date_list
	print(col)
	data =pd.Series(
		[df.loc[(df[col] == area)][date].sum() for date in time_series_date_list],
		)
	return data


In [ ]:
data_by_area(area='US', df=confirmed).plot()

In [ ]:
from functions import make_country_labels, make_state_labels

from data import confirmed

data = confirmed.copy()
print(data.groupby('Country/Region').sum().iloc[:,-1].sort_values(ascending=False).index)


make_country_labels(data=confirmed)

In [ ]:
data.groupby('Country/Region').sum().iloc[:,-2].sort_values(ascending=False)

In [ ]:
import pandas as pd
def get_daily_reports(local=True):
    start_date = pd.to_datetime('01/22/2020')
    end_date = pd.to_datetime('today')
    dates = pd.date_range(start_date, end_date)
    daily_reports = {}
    all_reports = []
    if local == True:
        print('Getting daily reports using local data')
        for date in dates:
            date_str = date.strftime('%m-%d-%Y')
            file_name = 'data/' + date_str + '.csv'
            try:
                df = pd.read_csv(file_name, header=0)
                df['Date'] = date_str
                all_reports.append(df)
            except:
                print("Failed to load {file}".format(file=file_name))
    elif local == False:
        print('Getting daily reports using on-line data')
        daily_report_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'
        for date in dates:
            # File format: 'MM-DD-YYYY.csv'
            date_str = date.strftime('%m-%d-%Y')
            file_name =date_str + '.csv'
            url = daily_report_url + file_name
            try:
                df = pd.read_csv(file_name, header=0)
                df['Date'] = date_str
                all_reports.append(df)
            except:
                print("Failed to load {file}".format(file=file_name))
    daily_reports = pd.concat(all_reports, axis=0, ignore_index=True)
#    valid_dates = [pd.to_datetime(date) for date in list(daily_reports.keys())]
    return daily_reports

In [ ]:
df = get_daily_reports()

df['Date']
df.Date = pd.to_datetime(df.Date)
#df['Last Update'] = pd.to_datetime(df['Last Update'])
#df['Last Update']

In [ ]:
print(df.columns)
area = 'US'
most_recent = df[df['Country/Region'] == area]['Last Update'].max() - pd.Timedelta('12 hours')
df[(df['Country/Region'] == area) & (df['Last Update'] >= most_recent)].sort_values('Last Update', ascending=False)

In [ ]:
def recent_updates(area='US', timedelta='12 hours', sort='Confirmed', df=None):
    ascending = {
        'Province/State': True
    }
    most_recent = df[df['Country/Region'] == area]['Last Update'].max() - pd.Timedelta(timedelta)
    return df[
        (df['Country/Region'] == area) & (df['Last Update'] >= most_recent)].sort_values(
            sort, ascending=ascending.get(sort, False))


In [ ]:
import dash_table

In [ ]:
from model import data_by_area
from data import time_series_dates, confirmed, get_states
import numpy as np

all_countries = confirmed.groupby('Country/Region').sum()

pandemic = 100
pandemic_countries = all_countries[all_countries['3/13/20'] >= pandemic].copy()

#confirmed[confirmed.columns[4:]].diff(axis=1)/confirmed[confirmed.columns[4:]].plot()

ndays = 7
data = pandemic_countries[pandemic_countries.columns[-ndays:-1]].copy()
data = data.sort_values(list(data.columns)[-1:], ascending=False)

rates = (data.diff(axis=1)/data)
rates = rates[rates.columns[1:]]
rates = rates.sort_values(list(rates.columns)[-1:], ascending=False)

from math import log
doubling_times = log(2)/rates
doubling_times[doubling_times == np.inf]=None
#print(doubling_times.mean(axis=1).sort_values())

get_states(df=confirmed)

In [4]:
import pandas as pd
from data import assign_state, confirmed, get_states
df = confirmed.copy()
df = get_states(df)
df['State'].unique()
#df['location'] = pd.DataFrame(df['Province/State'].apply(lambda x: assign_state(x)))
#df['location'].unique()

array(['nan', 'British Columbia', 'New South Wales', 'Victoria',
       'Queensland', 'South Australia', 'From Diamond Princess',
       'Western Australia', 'Tasmania', 'Northern Territory', 'Ontario',
       'Alberta', 'Quebec', 'Washington', 'New York', 'California',
       'Massachusetts', 'Diamond Princess', 'Grand Princess', 'Georgia',
       'Colorado', 'Florida', 'New Jersey', 'Oregon', 'Texas', 'Illinois',
       'Pennsylvania', 'Iowa', 'Maryland', 'North Carolina',
       'South Carolina', 'Tennessee', 'Virginia', 'Arizona', 'Indiana',
       'Kentucky', 'District of Columbia', 'Nevada', 'New Hampshire',
       'Minnesota', 'Nebraska', 'Ohio', 'Rhode Island', 'Wisconsin',
       'Connecticut', 'Hawaii', 'Oklahoma', 'Utah', 'Kansas', 'Louisiana',
       'Missouri', 'Vermont', 'Alaska', 'Arkansas', 'Delaware', 'Idaho',
       'Maine', 'Michigan', 'Mississippi', 'Montana', 'New Mexico',
       'North Dakota', 'South Dakota', 'West Virginia', 'Wyoming',
       'Hubei', 'France', 

In [27]:
from data import daily_report_data
most_recent = daily_report_data.groupby('Country/Region')['Last Update'].max()[area].to_pydatetime()

type(most_recent)

datetime.datetime

In [ ]:
import numpy as np
import pandas as pd
df = pd.DataFrame(np.random.randint(0,10,size=(10, 9)), columns=list('ABCDEFGHI'))
#print(df)
#print(df.diff(axis=1))
df[df.columns[-1:]]